# Project-1 - Yaohuang Liu + Xinshuang Zhao
----

## Main Function
- #### Read the 'ratings.csv' and remove the header.  
- #### Output(3 columns): userId, movieId, rating

`rtRDD` RDD for all ratings 

In [1]:
rtRDD=sc.textFile("ratings.csv") \
    .map(lambda row: row.split(","))
rtRDD_Header=rtRDD.first()
rtRDD=rtRDD.filter(lambda x: x!=rtRDD_Header).map(lambda row: (row[0],row[1],float(row[2])))
rtRDD.collect()

[('1', '31', 2.5),
 ('1', '1029', 3.0),
 ('1', '1061', 3.0),
 ('1', '1129', 2.0),
 ('1', '1172', 4.0),
 ('1', '1263', 2.0),
 ('1', '1287', 2.0),
 ('1', '1293', 2.0),
 ('1', '1339', 3.5),
 ('1', '1343', 2.0),
 ('1', '1371', 2.5),
 ('1', '1405', 1.0),
 ('1', '1953', 4.0),
 ('1', '2105', 4.0),
 ('1', '2150', 3.0),
 ('1', '2193', 2.0),
 ('1', '2294', 2.0),
 ('1', '2455', 2.5),
 ('1', '2968', 1.0),
 ('1', '3671', 3.0),
 ('2', '10', 4.0),
 ('2', '17', 5.0),
 ('2', '39', 5.0),
 ('2', '47', 4.0),
 ('2', '50', 4.0),
 ('2', '52', 3.0),
 ('2', '62', 3.0),
 ('2', '110', 4.0),
 ('2', '144', 3.0),
 ('2', '150', 5.0),
 ('2', '153', 4.0),
 ('2', '161', 3.0),
 ('2', '165', 3.0),
 ('2', '168', 3.0),
 ('2', '185', 3.0),
 ('2', '186', 3.0),
 ('2', '208', 3.0),
 ('2', '222', 5.0),
 ('2', '223', 1.0),
 ('2', '225', 3.0),
 ('2', '235', 3.0),
 ('2', '248', 3.0),
 ('2', '253', 4.0),
 ('2', '261', 4.0),
 ('2', '265', 5.0),
 ('2', '266', 5.0),
 ('2', '272', 3.0),
 ('2', '273', 4.0),
 ('2', '292', 3.0),
 ('2', '2

In [2]:
givenMovieId = '31'
givenUserId = '1'
avgRating = 3.0

---
- #### Apply filter to rtRDD, where 3rd column(rating) > avgRating and 2nd column(movieId) = givenMovieId
- #### Return all distinct userIds that meet the requirements

`GoodRtUsRDD` RDD for users that complement on the same movie

In [3]:
GoodRtUsRDD=rtRDD.filter(lambda row: row[2]>avgRating) \
     .filter(lambda row: row[1]==givenMovieId)\
     .map(lambda row: row[0]) \
     .distinct()
GoodRtUsLs=GoodRtUsRDD.collect() 
GoodRtUsLs

['73',
 '110',
 '111',
 '325',
 '341',
 '485',
 '496',
 '641',
 '31',
 '32',
 '165',
 '242',
 '487',
 '511',
 '516',
 '525',
 '607']

---
- #### For each userId that meets the requirements, get all the movieIds that user has seen and their corresponding ratings.

`UserSltMvRDD` RDD for all the movieIds that user has seen and their corresponding ratings

In [4]:
UserSltMvRDD=rtRDD.filter(lambda row: row[0] in GoodRtUsLs).map(lambda row: (row[1],row[2]))
UserSltMvRDD.collect()

[('31', 4.0),
 ('32', 4.5),
 ('50', 3.5),
 ('111', 4.5),
 ('260', 4.0),
 ('296', 4.5),
 ('318', 4.0),
 ('372', 3.5),
 ('379', 4.0),
 ('527', 5.0),
 ('541', 4.0),
 ('778', 3.5),
 ('858', 4.5),
 ('904', 4.0),
 ('1089', 4.5),
 ('1091', 3.0),
 ('1186', 2.0),
 ('1197', 4.5),
 ('1206', 4.5),
 ('1208', 4.5),
 ('1221', 4.5),
 ('1234', 4.5),
 ('1299', 4.0),
 ('1378', 4.0),
 ('1416', 3.0),
 ('1957', 4.5),
 ('1967', 4.5),
 ('2028', 4.5),
 ('2248', 4.0),
 ('2329', 4.5),
 ('2502', 4.0),
 ('2571', 4.5),
 ('2692', 4.5),
 ('2959', 5.0),
 ('2997', 4.5),
 ('3072', 3.0),
 ('3087', 3.5),
 ('3698', 4.5),
 ('3755', 4.0),
 ('3897', 4.0),
 ('3911', 4.0),
 ('4886', 4.0),
 ('4979', 3.5),
 ('4993', 4.5),
 ('5418', 4.5),
 ('5952', 4.5),
 ('5995', 3.5),
 ('6539', 3.5),
 ('6711', 4.5),
 ('7153', 4.5),
 ('7361', 5.0),
 ('8961', 4.0),
 ('33154', 4.0),
 ('33794', 4.5),
 ('44191', 4.0),
 ('44195', 4.0),
 ('48394', 4.5),
 ('50872', 4.5),
 ('54503', 4.0),
 ('55280', 4.5),
 ('55820', 4.0),
 ('58559', 5.0),
 ('59315', 5.0)

---
- #### Use aggregateByKey() to aggregate by movieId, and return the movie's average rating
- #### Sort average rating descending so that we get movies with highest rating in front.

`RecMvRDD` RDD for aggregated movie's average rating by movie Id

`ResultRDD` RDD for sorted result

In [6]:
RecMvRDD=UserSltMvRDD.aggregateByKey((0,0),
                                      lambda a, b: (a[0]+b, a[1]+1),
                                      lambda a, b: (a[0]+b[0], a[1]+b[1])) \
                                    .map(lambda row: (row[0],row[1][0]/row[1][1]))
ResultRDD=RecMvRDD.sortBy(lambda row: row[1], ascending=False)
ResultRDD.collect()

[('1201', 5.0),
 ('1204', 5.0),
 ('4519', 5.0),
 ('50641', 5.0),
 ('64839', 5.0),
 ('81564', 5.0),
 ('93855', 5.0),
 ('112852', 5.0),
 ('445', 5.0),
 ('932', 5.0),
 ('2330', 5.0),
 ('4973', 5.0),
 ('8645', 5.0),
 ('164', 5.0),
 ('348', 5.0),
 ('898', 5.0),
 ('900', 5.0),
 ('912', 5.0),
 ('933', 5.0),
 ('949', 5.0),
 ('953', 5.0),
 ('994', 5.0),
 ('1084', 5.0),
 ('1179', 5.0),
 ('1219', 5.0),
 ('1227', 5.0),
 ('1269', 5.0),
 ('1276', 5.0),
 ('1304', 5.0),
 ('1939', 5.0),
 ('1956', 5.0),
 ('1962', 5.0),
 ('1964', 5.0),
 ('2112', 5.0),
 ('2243', 5.0),
 ('2272', 5.0),
 ('2405', 5.0),
 ('2764', 5.0),
 ('2857', 5.0),
 ('2875', 5.0),
 ('2919', 5.0),
 ('3005', 5.0),
 ('3044', 5.0),
 ('3071', 5.0),
 ('3100', 5.0),
 ('3111', 5.0),
 ('3198', 5.0),
 ('3330', 5.0),
 ('3359', 5.0),
 ('3501', 5.0),
 ('5060', 5.0),
 ('17', 5.0),
 ('222', 5.0),
 ('835', 5.0),
 ('1301', 5.0),
 ('2527', 5.0),
 ('3635', 5.0),
 ('3729', 5.0),
 ('116', 5.0),
 ('562', 5.0),
 ('58559', 5.0),
 ('5244', 5.0),
 ('60293', 5.0),
 

---
- #### Open the 'movies.csv' file which include all movies' id & title.

`mvRDD` RDD for movies.csv

In [8]:
mvRDD=sc.textFile("movies.csv") \
    .map(lambda row: row.split(","))
mvRDD_Header=mvRDD.first()
mvRDD=mvRDD.filter(lambda x: x!=mvRDD_Header).map(lambda row: (row[0],row[1]))
mvRDD.collect()

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)'),
 ('4', 'Waiting to Exhale (1995)'),
 ('5', 'Father of the Bride Part II (1995)'),
 ('6', 'Heat (1995)'),
 ('7', 'Sabrina (1995)'),
 ('8', 'Tom and Huck (1995)'),
 ('9', 'Sudden Death (1995)'),
 ('10', 'GoldenEye (1995)'),
 ('11', '"American President'),
 ('12', 'Dracula: Dead and Loving It (1995)'),
 ('13', 'Balto (1995)'),
 ('14', 'Nixon (1995)'),
 ('15', 'Cutthroat Island (1995)'),
 ('16', 'Casino (1995)'),
 ('17', 'Sense and Sensibility (1995)'),
 ('18', 'Four Rooms (1995)'),
 ('19', 'Ace Ventura: When Nature Calls (1995)'),
 ('20', 'Money Train (1995)'),
 ('21', 'Get Shorty (1995)'),
 ('22', 'Copycat (1995)'),
 ('23', 'Assassins (1995)'),
 ('24', 'Powder (1995)'),
 ('25', 'Leaving Las Vegas (1995)'),
 ('26', 'Othello (1995)'),
 ('27', 'Now and Then (1995)'),
 ('28', 'Persuasion (1995)'),
 ('29', '"City of Lost Children'),
 ('30', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'),
 ('31', 

---
- #### Join (movieId, avgRating) with (movieId, title) and take 5 of them.

In [9]:
ResultRDD.join(mvRDD).map(lambda row: (row[1][1],row[1][0])).take(5)

[('Lawrence of Arabia (1962)', 5.0),
 ('"Wrestler', 5.0),
 ('Megamind (2010)', 5.0),
 ('Guardians of the Galaxy (2014)', 5.0),
 ('Fatal Instinct (1993)', 5.0)]

---
- ### Below is a function that integrates all the process above.
- #### You can pass & test with other values you want.

In [11]:
def recommend_function(givenMovieId = '31', givenUserId='1',avgRating=3.0):
    # Read in Rating records
    rtRDD=sc.textFile("ratings.csv") \
     .map(lambda row: row.split(","))
    
    # Take out header and filter first 3 columns for use
    rtRDD_Header=rtRDD.first()
    rtRDD=rtRDD.filter(lambda x: x!=rtRDD_Header).map(lambda row: (row[0],row[1],float(row[2])))
    
    # Filter distinct users with good rating on the given Movie
    GoodRtUsRDD=rtRDD.filter(lambda row: row[2]>avgRating) \
     .filter(lambda row: row[1]==givenMovieId)\
     .map(lambda row: row[0]) \
     .distinct()
    GoodRtUsLs=GoodRtUsRDD.collect()
    # Prepare User list to be Python list
    UserSltMvRDD=rtRDD.filter(lambda row: row[0] in GoodRtUsLs).map(lambda row: (row[1],row[2]))
    
    # AggregateByKey to compute average rate for each movie
    RecMvRDD=UserSltMvRDD.aggregateByKey((0,0),
     lambda a, b: (a[0]+b, a[1]+1),
     lambda a, b: (a[0]+b[0], a[1]+b[1])) \
     .map(lambda row: (row[0],row[1][0]/row[1][1]))
    ResultRDD=RecMvRDD.sortBy(lambda row: row[1], ascending=False)
    
    # Prepare Movie.csv for join
    mvRDD=sc.textFile("movies.csv") \
     .map(lambda row: row.split(","))
    mvRDD_Header=mvRDD.first()
    mvRDD=mvRDD.filter(lambda x: x!=mvRDD_Header).map(lambda row: (row[0],row[1]))
    
    # Join results with movie names
    return ResultRDD.join(mvRDD).map(lambda row: (row[1][1],row[1][0])).take(5)
print("Default example with movieId:'31', userId:'1', avgRating: 3.0 ")
print(recommend_function())
print("Another test example with movieId: '7', userId:'1', avgRating:2.5 ")
print(recommend_function('7','1',2.5))

Default example with movieId:'31', userId:'1', avgRating: 3.0 
[('Lawrence of Arabia (1962)', 5.0), ('"Wrestler', 5.0), ('Megamind (2010)', 5.0), ('Guardians of the Galaxy (2014)', 5.0), ('Fatal Instinct (1993)', 5.0)]
Another test example with movieId: '7', userId:'1', avgRating:2.5 
[('Crumb (1994)', 5.0), ('Three Colors: Blue (Trois couleurs: Bleu) (1993)', 5.0), ('Boys and Girls (2000)', 5.0), ('"Amazing Panda Adventure', 5.0), ('Diabolique (Les diaboliques) (1955)', 5.0)]
